In [1]:
import numpy as np
import pandas as pd

# Carica i dati pre-processati come fai in 02/03 (adatta se li salvi su pickle)
from joblib import load

X_train = load("data/processed/X_train.joblib")
X_test = load("data/processed/X_test.joblib")
y_train = load("data/processed/y_train.joblib")
y_test = load("data/processed/y_test.joblib")

mlp = load("models/mlp_burnout.pkl")      # il tuo MLP salvato
scaler = load("models/scaler.pkl")        # se ne usi uno
feature_names = X_train.columns


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/X_train.joblib'

In [ ]:
def what_if_scenario(model, scaler, x_row: pd.Series, deltas: dict):
    """
    x_row: una riga di X (Series)
    deltas: dict {feature_name: delta_value}
    """
    x_base = x_row.copy()

    x_new = x_row.copy()
    for feat, delta in deltas.items():
        if feat in x_new.index:
            x_new[feat] = x_new[feat] + delta

    # scaler opzionale
    if scaler is not None:
        x_base_scaled = scaler.transform([x_base.values])
        x_new_scaled = scaler.transform([x_new.values])
    else:
        x_base_scaled = [x_base.values]
        x_new_scaled = [x_new.values]

    base_pred = model.predict_proba(x_base_scaled)[0]
    new_pred = model.predict_proba(x_new_scaled)[0]

    return {
        "x_base": x_base,
        "x_new": x_new,
        "base_proba": base_pred,
        "new_proba": new_pred,
    }


Example down

In [ ]:
# prendi un caso ad alto burnout (classe 2)
mask_high = (y_test == 2)
idx = np.where(mask_high)[0][0]
x_example = X_test.iloc[idx]

deltas = {
    "sleep_hours": +1.0,
    "screen_time_hours": -2.0,
    "work_hours": -1.0,
}

result = what_if_scenario(mlp, scaler, x_example, deltas)

print("Base proba (low, medium, high):", result["base_proba"])
print("New  proba (low, medium, high):", result["new_proba"])
